In [1]:
import requests, boto3, pandas as pd, os, sys, pprint
from pathlib import Path
pp = pprint.PrettyPrinter(indent=2, compact=True, width=80)

from entities.RedditAccount import RedditAccount
from entities.DailyUpload import DailyUpload

#TODO: Change path accordingly in handler


# Making the current directory in which this file is in discoverable to python.
# Commenting it here because it will not work in jupyter notebook. It will work in lambda though.
#sys.path.append(os.path.join(os.path.dirname(__file__)))

# Below should be used only in jupyter notebook
sys.path.append('./')


REDDIT_AUTH_URL = 'https://www.reddit.com/api/v1/access_token'
REDDIT_ACCOUNTS_TABLE_NAME = 'RedditAccountsTable-dev'
DAILY_UPLOADS_TABLE = "DailyUploadsTable-dev"
REDDIT_API_URL_TOP = "https://oauth.reddit.com/r/placeholder_value/top"
REDDIT_API_URL_SORT = "https://oauth.reddit.com/r/placeholder_value/sort"

ddb = boto3.client("dynamodb", region_name="ap-south-1")
subreddit='funny'

## Get values from reddit accounts table

In [2]:
reddit_account = RedditAccount(subreddit=subreddit, ddb=ddb)
daily_upload = DailyUpload(subreddit=subreddit)



In [3]:
reddit_account.fetch_and_update_account_details(REDDIT_ACCOUNTS_TABLE_NAME)

## Authorise with reddit

In [4]:
reddit_account.authenticate_with_api()

In [5]:
res = requests.post(REDDIT_AUTH_URL, auth=reddit_account.auth, data=reddit_account.data, headers= reddit_account.headers)
# reddit_account.data
res

<Response [200]>

In [6]:
reddit_account.fetch_access_token(REDDIT_AUTH_URL)

## Send request for posts (with params, limit 100 posts)

In [7]:

REDDIT_API_URL_TOP = REDDIT_API_URL_TOP.replace("placeholder_value", subreddit)
top_posts = reddit_account.fetch_posts_as_json(REDDIT_API_URL_TOP, params = {'limit': '100'})


In [8]:
len(top_posts['data']['children'])

100

In [9]:
daily_upload.parse_posts(top_posts)

# Pushshift api tryout

In [32]:
api = PushshiftAPI()

In [157]:
from datetime import datetime, timedelta

In [158]:
today =  datetime.today()
yesterday = datetime.today() - timedelta(days=1)
day_before_yesterday = yesterday - timedelta(days=1)

In [159]:
today = datetime(today.year, today.month, today.day, 0,0,0).timestamp()
yesterday = datetime(yesterday.year, yesterday.month, yesterday.day,0,0,0).timestamp()
day_before_yesterday = datetime(day_before_yesterday.year, day_before_yesterday.month, day_before_yesterday.day,0,0,0).timestamp()

In [92]:
 a = list(api.search_submissions(after=day_before_yesterday, before=today, subreddit='funny', filter=['url', 'title'], limit = 10))

KeyboardInterrupt: 

In [201]:
int(day_before_yesterday)

1628447400

In [235]:
def get_pushshift_data(data_type, **kwargs):
    """
    Gets data from the pushshift api.
 
    data_type can be 'comment' or 'submission'
    The rest of the args are interpreted as payload.
 
    Read more: https://github.com/pushshift/api
    """
 
    base_url = f"https://api.pushshift.io/reddit/search/submission/?subreddit=funny&num_comments=>0&after={int(day_before_yesterday)}&before={int(yesterday)}&is_video=true&sort_type=score&sort=score:asc&size=100&aggs=subreddit"
#     payload = {}
#     print(payload)
    request = requests.get(base_url)
    return request.json()

In [236]:
data_type="submission"     # give me comments, use "submission" to publish something
query="funny"          # Add your query
duration="1d"          # Select the timeframe. Epoch value or Integer + "s,m,h,d" (i.e. "second", "minute", "hour", "day")
size=1000               # maximum 1000 comments
sort_type="score"       # Sort by score (Accepted: "score", "num_comments", "created_utc")
sort="desc"             # sort descending
aggs="subreddit"        #"author", "link_id", "created_utc", "subreddit"

In [254]:
base_url = f"https://api.pushshift.io/reddit/search/submission/?subreddit=funny&num_comments=>0&over_18=false&after={int(day_before_yesterday)}&before={int(yesterday)}&is_video=true&sort_type=score&sort=score:desc&size=100&aggs=subreddit"

request = requests.get(base_url)
b = request.json()

In [263]:
b['data'][59]

{'all_awardings': [],
 'allow_live_comments': False,
 'author': 'THCv1',
 'author_flair_css_class': None,
 'author_flair_richtext': [],
 'author_flair_text': None,
 'author_flair_type': 'text',
 'author_fullname': 't2_1bfzjru7',
 'author_is_blocked': False,
 'author_patreon_flair': False,
 'author_premium': False,
 'awarders': [],
 'can_mod_post': False,
 'contest_mode': False,
 'created_utc': 1628530642,
 'domain': '/r/funny/comments/p16ap5/we_all_have_at_least_one_in_every_town/',
 'full_link': 'https://www.reddit.com/r/funny/comments/p16ap5/we_all_have_at_least_one_in_every_town/',
 'gildings': {},
 'id': 'p16ap5',
 'is_created_from_ads_ui': False,
 'is_crosspostable': True,
 'is_meta': False,
 'is_original_content': False,
 'is_reddit_media_domain': False,
 'is_robot_indexable': True,
 'is_self': False,
 'is_video': True,
 'link_flair_background_color': '',
 'link_flair_richtext': [],
 'link_flair_text_color': 'dark',
 'link_flair_type': 'text',
 'locked': False,
 'media_only': Fal

In [238]:
for post in a['data']:    
    pp.pprint(post)
    break

   
    
        


{ 'all_awardings': [],
  'allow_live_comments': False,
  'author': 'Tinzeii',
  'author_flair_css_class': None,
  'author_flair_richtext': [],
  'author_flair_text': None,
  'author_flair_type': 'text',
  'author_fullname': 't2_4fknz7x5',
  'author_is_blocked': False,
  'author_patreon_flair': False,
  'author_premium': False,
  'awarders': [],
  'can_mod_post': False,
  'contest_mode': False,
  'created_utc': 1628460924,
  'domain': '/r/funny/comments/p0o96c/enjoy_ittt/',
  'full_link': 'https://www.reddit.com/r/funny/comments/p0o96c/enjoy_ittt/',
  'gildings': {},
  'id': 'p0o96c',
  'is_created_from_ads_ui': False,
  'is_crosspostable': True,
  'is_meta': False,
  'is_original_content': False,
  'is_reddit_media_domain': False,
  'is_robot_indexable': True,
  'is_self': False,
  'is_video': True,
  'link_flair_background_color': '',
  'link_flair_richtext': [],
  'link_flair_text_color': 'dark',
  'link_flair_type': 'text',
  'locked': False,
  'media_only': False,
  'no_follow': Tr

In [239]:
post

{'all_awardings': [],
 'allow_live_comments': False,
 'author': 'Tinzeii',
 'author_flair_css_class': None,
 'author_flair_richtext': [],
 'author_flair_text': None,
 'author_flair_type': 'text',
 'author_fullname': 't2_4fknz7x5',
 'author_is_blocked': False,
 'author_patreon_flair': False,
 'author_premium': False,
 'awarders': [],
 'can_mod_post': False,
 'contest_mode': False,
 'created_utc': 1628460924,
 'domain': '/r/funny/comments/p0o96c/enjoy_ittt/',
 'full_link': 'https://www.reddit.com/r/funny/comments/p0o96c/enjoy_ittt/',
 'gildings': {},
 'id': 'p0o96c',
 'is_created_from_ads_ui': False,
 'is_crosspostable': True,
 'is_meta': False,
 'is_original_content': False,
 'is_reddit_media_domain': False,
 'is_robot_indexable': True,
 'is_self': False,
 'is_video': True,
 'link_flair_background_color': '',
 'link_flair_richtext': [],
 'link_flair_text_color': 'dark',
 'link_flair_type': 'text',
 'locked': False,
 'media_only': False,
 'no_follow': True,
 'num_comments': 7,
 'num_cros